## Wednesday, April 24, 2024

mamba activate llama3

[pip trl](https://pypi.org/project/trl/)

This is a basic example of how to use the PPOTrainer from the library. Based on a query the language model creates a response which is then evaluated. The evaluation could be a human in the loop or another model's output.

This all runs, but I don't really see what it is doing. meh ...

In [1]:
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
# Yup, still getting errors complaining about the 4090 ... 
#   Using RTX 4000 series doesn't support faster communication broadband via P2P or IB. 
#   Please set `NCCL_P2P_DISABLE="1"` and `NCCL_IB_DISABLE="1" or use `accelerate launch` which will do this automatically.

import os

os.environ["NCCL_P2P_DISABLE"]="1"
os.environ["NCCL_IB_DISABLE"]="1"

In [3]:
# imports
import torch
from transformers import AutoTokenizer
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model
from trl.core import respond_to_batch

In [4]:
# get models
model = AutoModelForCausalLMWithValueHead.from_pretrained('gpt2')
model_ref = create_reference_model(model)

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

In [6]:
# initialize trainer
ppo_config = PPOConfig(batch_size=1, mini_batch_size=1)

In [7]:
# encode a query
query_txt = "This morning I went to the "
query_tensor = tokenizer.encode(query_txt, return_tensors="pt")

In [8]:
# get model response
response_tensor  = respond_to_batch(model, query_tensor)

In [9]:
# create a ppo trainer
ppo_trainer = PPOTrainer(ppo_config, model, model_ref, tokenizer)

/home/rob/miniforge3/envs/llama3/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:262: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(


In [10]:
# define a reward for response
# (this could be any reward such as human feedback or output from another model)
reward = [torch.tensor(1.0)]

In [11]:
# train model for one step with ppo
train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)

/home/rob/miniforge3/envs/llama3/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:1285: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /opt/conda/conda-bld/pytorch_1704987615470/work/aten/src/ATen/native/ReduceOps.cpp:1760.)
  std_scores = data["scores"].std()
/home/rob/miniforge3/envs/llama3/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:1312: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /opt/conda/conda-bld/pytorch_1704987615470/work/aten/src/ATen/native/ReduceOps.cpp:1760.)
  stats["tokens/queries_len_std"] = torch.std(query_lens).cpu().numpy().item()
/home/rob/miniforge3/envs/llama3/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:1315: UserWarning: std(): degrees of freedom is <= 0. Correction should be stric